In [ ]:
from sklearn.pipeline import make_pipeline
import numpy as np
from dense_subgraph import sdp, qp
import utils
from cs_classification import cs_p1_graphs_to_points, cs_p2_graphs_to_points
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.metrics import classification_report

In [ ]:
# Read brain graph files into numpy arrays
graphs_A = utils.get_graphs_from_files("./datasets/children/asd/")
graphs_B = utils.get_graphs_from_files("./datasets/children/td/")

graphs, labels = utils.label_and_concatenate_graphs(graphs_A, graphs_B, a_label="ASD", b_label="TD")

In [ ]:
class ContrastSubgraphTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, a_label=None, b_label=None,
                    alpha=None, alpha2=None,
                    percentile=None, percentile2=None,
                    problem=None, solver=None, num_cs=None) -> None:
        self.a_label = a_label
        self.b_label = b_label

        self.alpha = alpha
        self.alpha2 = alpha2 or alpha
        self.percentile = percentile
        self.percentile2 = percentile2 or percentile
        
        self.problem = problem
        self.solver = solver
        self.num_cs = num_cs

        self.cs_a_b_list = []
        self.cs_b_a_list = []
        self.cs_list = []

    def fit(self, X, y=None):
         # Create and Write Summary Graphs
        self.summary_A = utils.summary_graph(X[np.where(y == self.a_label)])
        self.summary_B = utils.summary_graph(X[np.where(y == self.b_label)])

        if self.problem == 1:
            self.find_cs_p1()
        else: # problem == 2
            self.find_cs_p2()
        
        return self

    def find_cs_p1(self) -> None:
        diff_a_b = self.summary_A - self.summary_B
        diff_b_a = self.summary_B - self.summary_A

        alpha_provided = bool(self.alpha)

        nodes = np.arange(diff_a_b.shape[0])
        node_mask_a_b = np.array([True]*nodes.shape[0])
        node_mask_b_a = np.array([True]*nodes.shape[0])

        for i in range(self.num_cs):
            masked_diff_a_b = utils.induce_subgraph(diff_a_b, nodes[node_mask_a_b])
            masked_diff_b_a = utils.induce_subgraph(diff_b_a, nodes[node_mask_b_a])

            # If no alpha value is provided, find the appropriate alpha value using the given percentile
            if not alpha_provided:
                # A -> B
                flat = masked_diff_a_b[np.triu_indices_from(masked_diff_a_b, k=1)]
                self.alpha = np.percentile(flat, self.percentile)
                print("alpha = {} ({}-th percentile)".format(self.alpha, self.percentile), end=", ")

                # B -> A
                flat = masked_diff_b_a[np.triu_indices_from(masked_diff_b_a, k=1)]
                self.alpha2 = np.percentile(flat, self.percentile2)
                print("alpha2 = {} ({}-th percentile)".format(self.alpha2, self.percentile2))

            cs_a_b = nodes[node_mask_a_b][self.solver(masked_diff_a_b, self.alpha)]
            self.cs_a_b_list.append(cs_a_b)
            cs_b_a = nodes[node_mask_b_a][self.solver(masked_diff_b_a, self.alpha2)]
            self.cs_b_a_list.append(cs_b_a)
            # Do not consider the previously found contrast subgraph nodes for future contrast subgraphs
            node_mask_a_b[cs_a_b] = False
            node_mask_b_a[cs_b_a] = False

            if len(nodes[node_mask_a_b]) == 0:
                print("Every node in the graph is included by a contrast subgraph(A->B)!\n\
                    Stopped at Contrast Subgraph {}.".format(i+1))
                break
            if len(nodes[node_mask_b_a]) == 0:
                print("Every node in the graph is included by a contrast subgraph (B->A)!\n\
                    Stopped at Contrast Subgraph {}.".format(i+1))
                break

    def find_cs_p2(self) -> None:
        diff = abs(self.summary_A - self.summary_B)

        alpha_provided = bool(self.alpha)

        nodes = np.arange(diff.shape[0])
        node_mask = np.array([True]*nodes.shape[0])
        
        for i in range(self.num_cs):
            masked_diff = utils.induce_subgraph(diff, nodes[node_mask])

            # If no alpha value is provided, find the appropriate alpha value using the given percentile
            if not alpha_provided:
                flat = masked_diff[np.triu_indices_from(masked_diff, k=1)]
                self.alpha = np.percentile(flat, self.percentile)
                print("alpha = {} ({}-th percentile)".format(self.alpha, self.percentile))
            
            cs = nodes[node_mask][self.solver(masked_diff, self.alpha)]
            self.cs_list.append(cs)
            node_mask[cs] = False

            if len(nodes[node_mask]) == 0:
                print("Every node in the graph is included by a contrast subgraph!\n\
                    Stopped at Contrast Subgraph {}.".format(i+1))
                break

    def transform(self, X):
        points = np.zeros((X.shape[0], 2))

        if self.problem == 1:
            num_cs = min(len(self.cs_a_b_list), len(self.cs_b_a_list))
            for i in range(num_cs):
                points += cs_p1_graphs_to_points(graphs=X,
                                                cs_a_b=self.cs_a_b_list[i],
                                                cs_b_a=self.cs_b_a_list[i])
        else: # problem == 2
            for i in range(len(self.cs_list)):
                points += cs_p2_graphs_to_points(graphs=X,
                                                contrast_subgraph=self.cs_list[i],
                                                summary_A=self.summary_A,
                                                summary_B=self.summary_B)
        return points

In [ ]:
class DiscriminativeEdgesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, a_label: str, b_label: str, num_edges: int) -> None:
        self.a_label = a_label
        self.b_label = b_label

        self.num_edges = num_edges

        self.axes_labels = [f"% similarity between important {a_label} edges",
                            f"% similarity between important {b_label} edges",
                            f"% similarity of whole graph with {a_label} class"]

    def fit(self, graphs, labels) -> None:
        # Create and Write Summary Graphs
        summary_A = utils.summary_graph(graphs[np.where(labels == self.a_label)])
        summary_B = utils.summary_graph(graphs[np.where(labels == self.b_label)])
            
        # Get the difference network between the edge weights in group A and B
        # Note that (u,v) is the same as (v,u), so we extract the upper triangle of the matrix
        self.diff_net = np.triu(summary_A - summary_B, k=1)

        # Find the num_edges most positive and most negative edge diffs
        partitions = np.argpartition(self.diff_net, (self.num_edges, -self.num_edges), axis=None)
        top_n = np.unravel_index(partitions[-self.num_edges:], self.diff_net.shape)
        bottom_n = np.unravel_index(partitions[:self.num_edges], self.diff_net.shape)

        # Ensure the top edges are all positive and the bottom edges are all negative
        top_edges = self.diff_net[top_n]
        positive = top_edges > 0
        self.positive_indices = (top_n[0][positive], top_n[1][positive])
        self.important_a_edges = self.diff_net[self.positive_indices]

        bottom_edges = self.diff_net[bottom_n]
        negative = bottom_edges < 0
        self.negative_indices = (bottom_n[0][negative], bottom_n[1][negative])
        self.important_b_edges = self.diff_net[self.negative_indices]

        self.a_sum = np.sum(self.important_a_edges)
        self.b_sum = np.sum(self.important_b_edges)
        self.full_sum = np.sum(np.abs(self.diff_net))

        return self

    
    def transform(self, graph):
        points = np.array(list(map(self.graph_to_point, graphs)))
        return points

    def graph_to_point(self, graph):
        graph[np.where(graph==0)] = -1
        return np.array([100*np.dot(self.important_a_edges, graph[self.positive_indices])/self.a_sum,
                         100*np.dot(self.important_b_edges, graph[self.negative_indices])/self.b_sum,
                         100*np.sum(np.multiply(graph, self.diff_net))/self.full_sum])

In [ ]:
import nested_cv
# Set up possible values of parameters to optimize over
p_grid = {"SVC": {"C": [0.1, 1], "gamma": [0.00001, 0.0001]},
          "ContrastSubgraphTransformer": {
            "a_label": ["ASD"],
            "b_label": ["TD"],
            "alpha": [None, 0.01],
            "alpha2": [None, 0.02],
            "percentile": [77],
            "percentile2": [79],
            "solver": [sdp],
            "problem": [1],
            "num_cs": [1],
            }
          }

pipe = [ContrastSubgraphTransformer, StandardScaler, SVC]


inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

results = nested_cv.classify(X=graphs, y=labels, pipeline_steps=pipe, step_params=p_grid, outer_cv=outer_cv, inner_cv=inner_cv)
results

In [11]:
x = np.array([ np.array([_, 2*_]) for _ in range(23)])
print(x)
StandardScaler().fit(x).transform(x)

[[ 0  0]
 [ 1  2]
 [ 2  4]
 [ 3  6]
 [ 4  8]
 [ 5 10]
 [ 6 12]
 [ 7 14]
 [ 8 16]
 [ 9 18]
 [10 20]
 [11 22]
 [12 24]
 [13 26]
 [14 28]
 [15 30]
 [16 32]
 [17 34]
 [18 36]
 [19 38]
 [20 40]
 [21 42]
 [22 44]]


array([[-1.6583124 , -1.6583124 ],
       [-1.50755672, -1.50755672],
       [-1.35680105, -1.35680105],
       [-1.20604538, -1.20604538],
       [-1.05528971, -1.05528971],
       [-0.90453403, -0.90453403],
       [-0.75377836, -0.75377836],
       [-0.60302269, -0.60302269],
       [-0.45226702, -0.45226702],
       [-0.30151134, -0.30151134],
       [-0.15075567, -0.15075567],
       [ 0.        ,  0.        ],
       [ 0.15075567,  0.15075567],
       [ 0.30151134,  0.30151134],
       [ 0.45226702,  0.45226702],
       [ 0.60302269,  0.60302269],
       [ 0.75377836,  0.75377836],
       [ 0.90453403,  0.90453403],
       [ 1.05528971,  1.05528971],
       [ 1.20604538,  1.20604538],
       [ 1.35680105,  1.35680105],
       [ 1.50755672,  1.50755672],
       [ 1.6583124 ,  1.6583124 ]])

In [ ]:
# # Set up possible values of parameters to optimize over
# p_grid = {"svc__C": [0.1, 1],
#           "svc__gamma": [0.00001, 0.0001],
#           "contrastsubgraphtransformer__a_label": ["ASD"],
#           "contrastsubgraphtransformer__b_label": ["TD"],
#           "contrastsubgraphtransformer__alpha": [None, 0.01],
#           "contrastsubgraphtransformer__alpha2": [None, 0.02],
#           "contrastsubgraphtransformer__percentile": [77],
#           "contrastsubgraphtransformer__percentile2": [79],
#           "contrastsubgraphtransformer__solver": [sdp],
#           "contrastsubgraphtransformer__problem": [1],
#           "contrastsubgraphtransformer__num_cs": [1],
#           }

# pipe = make_pipeline(ContrastSubgraphTransformer(), StandardScaler(), SVC())


# inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# # Nested CV with parameter optimization
# clf = GridSearchCV(estimator=pipe, param_grid=p_grid, cv=inner_cv)
# # nested_score = cross_validate(clf, X=graphs, y=labels, cv=outer_cv, return_estimator=True)
# X_train, X_test, y_train, y_test = train_test_split(graphs, labels, train_size=0.8, shuffle=True, random_state=42)
# clf.fit(X_train, y_train)
# print(classification_report(y_true=y_test, y_pred=clf.predict(X_test)))

In [ ]:
# Set up possible values of parameters to optimize over
# p_grid = [{"svc__C": [0.1, 1],
#           "svc__gamma": [0.00001, 0.0001],
#           "discriminativeedgestransformer__num_edges": [5, 6, 7]},
#           ]

# pipe = make_pipeline(DiscriminativeEdgesTransformer(a_label="ASD", b_label="TD", num_edges=6), StandardScaler(), SVC())


# inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# # Nested CV with parameter optimization
# clf = GridSearchCV(estimator=pipe, param_grid=p_grid, cv=inner_cv)
# nested_score = cross_validate(clf, X=graphs, y=labels, cv=outer_cv)
# nested_score, clf.get_params(deep=True)